In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
trainingDataFilename = 'trainingSet.csv'
testDataFilename = 'testSet.csv'
modelIdx = 1

In [3]:
def sigmoid(scores):
    predictions = np.zeros(len(scores))
    for i in range(len(predictions)):
        if scores[i] >= 0:
            predictions[i] +=  1.0 / (1.0 + np.exp(-scores[i]))
        else:
            predictions[i] += np.exp(scores[i]) / (1.0 + np.exp(scores[i]))
    return predictions

def lr(trainingSet, testSet):
    print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.01
    
    max_iterations = 500
    tol = 1e-6
    
    count = 0
    
    train_labels = trainingSet['decision']    
    trainingSet = trainingSet.drop('decision', axis=1)
    
    #print train_labels, trainingSet
    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    #X = np.concatenate((X, intercept.T), axis=1)
    X = np.hstack((X, intercept))
    diff = 100.0
    
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        scores = np.dot(X, w)
        predictions = sigmoid(scores)

        gradient = np.dot(X.T, (predictions - Y))

        for j in range(len(w)):
            gradient[j] += regularization * w[j]
            
        #gradient /= len(train_labels)
        w -= step_size * gradient
        norm_new = np.linalg.norm(w)
        
        diff = abs(norm_new - norm_old)
        #print w
        print count, diff
    
    return w

In [4]:
def svm(trainingSet, testSet):
    #print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.50
    
    max_iterations = 500
    tol = 1e-6
    #print len(trainingSet[trainingSet['decision'] == 1])
    count = 0
    train_labels = trainingSet['decision']    
    trainingSet = trainingSet.drop('decision', axis=1)

    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    #print train_labels
    for i in range(len(Y)):
        if Y[i] == 0:
            Y[i] = -1.0
        else:
            Y[i] = 1.0
    #print Y.tolist()
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))
    diff = 100.0
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        predictions = np.dot(X, w)
        #print X.shape, w.shape,predictions.shape
#         p = []
#         for i in range(0, 5200):
#             p.append(sum([w[j] * X[i][j] for j in range(len(trainingSet.columns))]))
        
#         print predictions, p 
        #print len(predictions)
        #print np.dot(w, X[0]), predictions[0]     
        error = 0
        gradient = np.zeros(len(w))
        for i in range(len(predictions)):
            if predictions[i] * Y[i] < 1.0:
                error += 1
                #gradient -= 1.0 * Y[i] * X[i]
                gradient -= np.multiply(X[i], Y[i])
            
        gradient /= 1.0 * len(train_labels)
        #print gradient.shape, X[0].shape
        
        for j in range(1, len(gradient)):
            gradient[j] += 1.0 * regularization * w[j]

        w -= 1.0 * step_size * gradient
        norm_new = np.linalg.norm(w)
        diff = abs(norm_new - norm_old)
        print count, diff, error
    print w
    return w

In [5]:
def get_accuracy_lr(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))
    
    scores = np.dot(X, w)
    predictions = sigmoid(scores)
        
    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(train_labels[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy LR:', '%.2f' % training_accuracy, count_train, total_train
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))

    scores = np.dot(X, w)
    predictions = sigmoid(scores)

    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(test_labels[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy LR:', '%.2f' % test_accuracy, count_test, total_test
    
def get_accuracy_svm(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))
    
    predictions = np.dot(X, w)
    
    for i in range(len(predictions)):
        if predictions[i] > 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(Y[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy SVM:', '%.2f' % training_accuracy
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))

    predictions = np.dot(X, w)

    for i in range(len(predictions)):
        if predictions[i] > 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(Y[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy SVM:', '%.2f' % test_accuracy

In [6]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

if modelIdx == 1:
    w = lr(trainingSet, testSet)
    get_accuracy_lr(w, trainingSet, testSet)
else:
    w = svm(trainingSet, testSet)
    get_accuracy_svm(w, trainingSet, testSet)

261
1 173.62870083173343
2 861.6362589116515
3 514.3000810583615
4 203.31923572662765
5 143.94747893059412
6 400.6045235190568
7 747.8017883868573
8 823.1386245055446
9 851.5270631911828
10 793.9097846637098
11 517.7003425288688
12 375.10135907427446
13 717.807882040491
14 724.2285001428148
15 44.22201783018511
16 469.55638917481815
17 196.29164044903644
18 13.173577571099031
19 298.4313223228404
20 371.1913246406658
21 697.5184438156001
22 633.2897973207337
23 394.8018895610926
24 147.74005140334202
25 23.767432251975833
26 254.98134342408935
27 283.56147688399255
28 579.3961719804336
29 508.653253932491
30 178.06373467519552
31 133.03618854799197
32 166.44983703210642
33 443.95142198039775
34 391.4163546368418
35 27.89584776485117
36 214.2035476520948
37 6.275498911346631
38 37.594626852847114
39 286.13817086707786
40 256.44120187032877
41 364.97420707517676
42 305.48803121336573
43 115.56250409161999
44 7.842955742645017
45 240.354360767569
46 209.23154827991902
47 334.9555998259325

378 2.7638991666171933
379 7.138066353828435
380 2.7524188763700295
381 7.097540399291574
382 2.748147964985037
383 7.163172914810275
384 2.696990681495663
385 6.9500325957142195
386 2.8646668580231562
387 6.962718646667781
388 2.806582785552564
389 6.9246811606562915
390 2.7472611887751555
391 6.807207169562389
392 2.570521403105886
393 6.703232563576421
394 2.570384091682172
395 6.554744817620303
396 2.613960665040395
397 6.744924934165283
398 2.556749844126898
399 6.53785759051425
400 2.598156933120663
401 6.579822677192169
402 2.631762003201402
403 6.663055523360526
404 2.5626665437348493
405 6.457288009297372
406 2.5409058739442116
407 6.320870624067538
408 2.5679148517911017
409 6.331494375098373
410 2.5714016397760133
411 6.269826506400932
412 2.524927365446274
413 6.199381812904903
414 2.4954244829104937
415 6.186484083332289
416 2.4736100417876514
417 6.08196974722614
418 2.408768986681025
419 6.007811280650458
420 2.496938058411615
421 6.039474895105741
422 2.4792221349816828